# 数据载入、存储及文件格式

## 1、文本格式数据的读写

In [2]:
'''
利用
read_csv    分隔符默认是','      它可以自动推断读取数据的类型
或read_table  分隔符默认是'\t'制表符
来读取数据
'''
import pandas as pd
import numpy as np
pd.options.display.max_rows=10
data=pd.read_csv('example/ex1.csv') 
print(data)                                            #读取到的数据默认认为第一行是表头行
data1=pd.read_table('example/ex1.csv',sep=',')        #需指定分隔符为','
print(data1)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [32]:
#对于不带表头行的文件
data3=pd.read_csv('example/ex2.csv',header=None)                         #指明没有表头行，默认自动分配表头
print(data3)
data4=pd.read_csv('example/ex2.csv',names=['a','b','c','d','message'])    #也可以自己指定
print(data4)

   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [37]:
#可以使得数据中某一列值成为返回的DataFrame的索引
names=list(data4.columns)
data5=pd.read_csv('example/ex2.csv',names=names)
print(data5)
data6=pd.read_csv('example/ex2.csv',names=names,index_col='message')  #利用index_col参数
print(data6)
data6=pd.read_csv('example/ex2.csv',names=names,index_col=3)          #直接传入列的序号也是可以的
print(data6)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
         a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12
    a   b   c message
d                    
4   1   2   3   hello
8   5   6   7   world
12  9  10  11     foo


In [39]:
#传入多个列名，可以实现分层索引
data1=pd.read_csv('example/ex3.csv',index_col=['key1','key2'])    
print(data1)

           value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16


In [41]:
#当文件中利用不同数量的空格隔开时，可以将分隔符设定为'\s+'
data2=pd.read_csv('example/ex3.txt',sep='\s+')
print(data2)                                              #由于行名的数量比数据列数少一个，所以第一列被推断为索引

            A         B         C
aaa -0.200000 -1.026000 -0.619000
bbb  0.927272  0.302900 -0.032600
ccc -0.264580 -0.386314 -0.217601
ddd -0.871800 -0.348350  1.100049


In [42]:
#skiprows可以用来跳过一些行读取
data3=pd.read_csv('example/ex4.csv',skiprows=[0,2,3])
print(data3)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [46]:
#缺失值处理
data4=pd.read_csv('example/ex5.csv')       #缺失的地方为NA或NULL，或直接为空，则会直接填充'NaN'
print(data4)
data4=pd.read_csv('example/ex5.csv',na_values=['iop',1])     #可以设定标识值，遇到标识值，同样填充'NaN'
print(data4)
sentinels={'message':['foo','NA'],'something':['two']}
data4=pd.read_csv('example/ex5.csv',na_values=sentinels)     #还可以对不同行采取不同的缺失值标识
print(data4)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
  something    a   b     c   d message
0       one  NaN   2   3.0   4     NaN
1       two  5.0   6   NaN   8   world
2     three  9.0  10  11.0  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


In [47]:
'''
read_csv中有非常非常非常多可选的参数，具体看书中P170
'''

'\nread_csv中有非常非常非常多可选的参数，具体看书中P170\n'

### 1.1、分块读入文本文件

In [48]:
pd.options.display.max_rows=10   #设置pandas的显示设置，最大显示行数为10

In [58]:
#当只想读取一个大文件的一小部分行时，可以指明nrows
data1=pd.read_csv('example/ex6.csv')
print(data1)
data1=pd.read_csv('example/ex6.csv',nrows=5)
print(data1)

    a   b   c   d message
0   1   2   3   4   hello
1   5   6   7   8   world
2   9  10  11  12     foo
3  13  14  15  16     uio
4  17  18  19  20     iop
5  21  22  23  24     yui
    a   b   c   d message
0   1   2   3   4   hello
1   5   6   7   8   world
2   9  10  11  12     foo
3  13  14  15  16     uio
4  17  18  19  20     iop


In [67]:
#利用chunksize分块遍历文件
chunker=pd.read_csv('example/ex6.csv',chunksize=2)
chunker

In [72]:
chunker=pd.read_csv('example/ex6.csv',chunksize=2)
tot=pd.Series([])
for pieces in chunker:                                                      #分块计算每块各自'message'不同信息出现的次数，再逐块相加
    tot=tot.add(pieces['message'].value_counts(),fill_value=0)
tot=tot.sort_values(ascending=False)
print(tot)

tot=pd.Series([])
for pieces in chunker:
    tot=tot.add(pieces['message'].value_counts(),fill_value=0)
tot=tot.sort_values(ascending=False)
print(tot)                                                                #chunker只能使用一次，类似生成器

message
foo      1.0
hello    1.0
iop      1.0
uio      1.0
world    1.0
yui      1.0
dtype: object
Series([], dtype: object)


### 1.2、将数据写入文本格式

In [4]:
data=pd.read_csv('example/ex1.csv',na_values='hello')
print(data)

   a   b   c   d message
0  1   2   3   4     NaN
1  5   6   7   8   world
2  9  10  11  12     foo


In [103]:
#利用DataFrame的to_csv方法可以将数据默认导出为以','分隔的文件  
data.to_csv('example/out.csv')          

In [101]:
!type example/ex1.csv              '''jupyter中的文件路径里,/,用来区分路径。但是windows系统的cmd里，用,\,来区分路径'''

命令语法不正确。


In [106]:
!type example\out.csv                     #注意，导出时连索引都导出去了

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [117]:
data1=pd.read_csv('example/out.csv')
print(data1)
data1=pd.read_csv('example/out.csv',index_col=0)
print(data1)

   Unnamed: 0  a   b   c   d message
0           0  1   2   3   4   hello
1           1  5   6   7   8   world
2           2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [5]:
#可以改变导出时的分隔符
import sys
data.to_csv(sys.stdout,sep='|')               #设置分隔符为'|'。写入到sys.stdout中，即控制台打印的文本

|a|b|c|d|message
0|1|2|3|4|
1|5|6|7|8|world
2|9|10|11|12|foo


In [7]:
#可以改变导出时的缺失值标注
data.to_csv(sys.stdout,na_rep='NULL')        #将缺失值标注为'NULL'

,a,b,c,d,message
0,1,2,3,4,NULL
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
#可以选择不导出索引或者标签
data.to_csv(sys.stdout,index=False,header=False)

1,2,3,4,
5,6,7,8,world
9,10,11,12,foo


In [9]:
#可以仅写入列的子集
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3
5,6,7
9,10,11


In [11]:
#Series也有to_csv方法
dates=pd.date_range('1/1/2000',periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [19]:
ts=pd.Series(np.arange(7),index=dates)
print(ts)
ts.to_csv('example/out2.csv',header=False)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32


In [20]:
!type example\out2.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 1.3、使用分隔格式

In [1]:
!type example\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [3]:
#对于任何有单字符分隔符的读取，都可以用内建csv模块读取
import csv
f=open('example/ex7.csv')
reader=csv.reader(f)
for line in reader:             #这样子读取出来的就没有引号。读取的是元组
    print(line)
    
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [5]:
#更进一步的处理
with open('example/ex7.csv') as f:
    lines=list(csv.reader(f))
header,values=lines[0],lines[1:]
data_dict={h:v for h,v in zip(header,zip(*values))}          #zip(*values)就是深入values中，将其中几个元组进行配对
print(data_dict)
b=list(zip(*values))
print(b)

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}
[('1', '1'), ('2', '2'), ('3', '3')]


In [6]:
#csv文件有各种各样的分隔符，可以设置csv.reader中的分隔符
reader=csv.reader(f,delimiter='|')     #表明不同元素间的分隔符为|

ValueError: I/O operation on closed file.

### 1.4、JSON数据

In [6]:
#使用json库来操作json数据
import json
with open('example/obj.json') as f:
    result=json.load(f)
print(result)
print(type(result))               #利用load直接将json文件中内容读取为字典


obj="""
{
    "name":"Wes",
    "places_lived":["United States","Spain","Germany"],
    "pet":null,
    "siblings":[{"name":"Scott","age":30,"pets":["Zeus","Zuko"]},
                {"name":"Katie","age":38,"pets":["Sixes","Stache","Cisco"]}
    ]
}
"""
result=json.loads(obj)                 #loads函数是load-str，读取字符串，将json字符串转换为字典
print(result)
print(type(result))

str1=json.dumps(result)                #dumps是dump-str，将字典转换为json字符串
print(str1)
print(type(str1))

with open('example/obj1.json','w') as f:
    json.dump({key:result[key] for key in result.keys() & {'name','places_lived'}},f)    #利用字典的切片，使用dump将字典对象直接输入到json文件中

{'name': 'Wes', 'places_lived': ['United States', 'Spain', 'Germany'], 'pet': None, 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']}, {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}
<class 'dict'>
{'name': 'Wes', 'places_lived': ['United States', 'Spain', 'Germany'], 'pet': None, 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']}, {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}
<class 'dict'>
{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}
<class 'str'>


In [7]:
#利用panda.read_json可以将json数据集按照次序转换为Series或DataFrame。默认假设时JSON数组中每个对象都是表里的一行
!type example\example.json

[
    {"a":1,"b":2,"c":3},
    {"a":4,"b":5,"c":6},
    {"a":7,"b":8,"c":9}
]


In [8]:
data=pd.read_json('example/example.json')
print(data)
print(type(data))

   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
<class 'pandas.core.frame.DataFrame'>


In [9]:
#利用Series和DataFrame的方法to_json可以将其数据导出为json字符串格式
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### 1.5、XML和HTML：网络抓取

In [10]:
#可以利用pandas.read_html函数来进行读取，会返回<table>标签中所有的表格型数据，返回DataFrame对象的列表
'''read_html所使用的附加库包括lxml、html5lib'''
tables=pd.read_html('example/fdic_failed_bank_list.html')
print(tables)
print(type(tables))
failures=tables[0]
print(failures)
print(type(failures))

[                             Bank Name             City  ST   CERT  \
0                          Allied Bank         Mulberry  AR     91   
1         The Woodbury Banking Company         Woodbury  GA  11297   
2               First CornerStone Bank  King of Prussia  PA  35312   
3                   Trust Company Bank          Memphis  TN   9956   
4           North Milwaukee State Bank        Milwaukee  WI  20364   
..                                 ...              ...  ..    ...   
542                 Superior Bank, FSB         Hinsdale  IL  32646   
543                Malta National Bank            Malta  OH   6629   
544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
545  National State Bank of Metropolis       Metropolis  IL   3815   
546                   Bank of Honolulu         Honolulu  HI  21029   

                   Acquiring Institution        Closing Date  \
0                           Today's Bank  September 23, 2016   
1                            U

In [17]:
#进一步的数据处理
close_timestamps=pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()          #求出来按年统计的关闭的企业数

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

#### 1.5.1、使用lxml.objectify解析XML

In [24]:
from lxml import objectify
path='example/mta_perf/Performance_MNR.xml'
parsed=objectify.parse(open(path))
root=parsed.getroot()                     #得到文件的根节点

data=[]
skip_fields=['PARENT_SEQ','INDICATOR_SEQ','DESIRED_CHANGE','DECIMAL_PLACES']     #定义一下需要跳过的标签
for elt in root.INDICATOR:                                                      #root_INDICATOR返回一个生成器，产生每一个<INDICATOR>XML元素
    el_data={}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag]=child.pyval
    data.append(el_data)
perf=pd.DataFrame(data)
print(perf)

              AGENCY_NAME                        INDICATOR_NAME  \
0    Metro-North Railroad  On-Time Performance (West of Hudson)   
1    Metro-North Railroad  On-Time Performance (West of Hudson)   
2    Metro-North Railroad  On-Time Performance (West of Hudson)   
3    Metro-North Railroad  On-Time Performance (West of Hudson)   
4    Metro-North Railroad  On-Time Performance (West of Hudson)   
..                    ...                                   ...   
643  Metro-North Railroad                Escalator Availability   
644  Metro-North Railroad                Escalator Availability   
645  Metro-North Railroad                Escalator Availability   
646  Metro-North Railroad                Escalator Availability   
647  Metro-North Railroad                Escalator Availability   

                                           DESCRIPTION  PERIOD_YEAR  \
0    Percent of commuter trains that arrive at thei...         2008   
1    Percent of commuter trains that arrive at thei..

In [23]:
list1=[{'a':1,'b':3,'c':4},{'b':2,'a':2,'d':1}]
print(pd.DataFrame(list1))

   a  b    c    d
0  1  3  4.0  NaN
1  2  2  NaN  1.0


## 2、二进制格式

In [25]:
#到pickle格式的转换。pandas对象有一个to_pickle方法可以将数据以pickle格式写入硬盘
frame=pd.read_csv('example/ex1.csv')
print(frame)
frame.to_pickle('example/frame_pickle')

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [26]:
#使用pandas.read_pickle读取pickle文件中pickle化的对象
pd.read_pickle('example/frame_pickle')
'''
但pickle仅推荐保存短期数据，因为其是底层的二进制数据，可能编译改变后，数据就变了
'''

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 2.1、使用HDF5格式

In [51]:
'''
HDF5格式广泛用于存储大量的科学数组数据
'''
frame=pd.DataFrame({'a':np.random.randn(100)})
store=pd.HDFStore('mydata.h5')                      #打开hdf5文件
store['obj1']=frame                                 #存储DataFrame对象
store['obj1_col']=frame['a']                        #存储Series对象
print(store)
print(type(store))

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

<class 'pandas.io.pytables.HDFStore'>


In [52]:
store['obj1_col']

0    -0.741889
1     0.576293
2    -1.444368
3     0.118887
4    -0.548854
        ...   
95   -0.410271
96    0.406931
97   -1.223729
98   -2.356708
99    1.027278
Name: a, Length: 100, dtype: float64

In [53]:
store.put('obj2',frame,format='table')          #等价于store['obj2']=frame，但是还设置了存储模式为'table'，这种模式支持下面的where查找方式
a=store.select('obj2',where=['index>=10 and index <=15'])
print(a)
store.close()                                   #记得关闭文件

           a
10  0.031739
11 -1.468505
12  0.991505
13 -0.450978
14  0.064991
15  0.316152


In [54]:
#还可以用pandas.read_hdf完成
frame.to_hdf('mydata.h5','obj3',format='table')

In [55]:
pd.read_hdf('mydata.h5','obj3','r+',where=['index < 5'])

,a
0,-0.741889
1,0.576293
2,-1.444368
3,0.118887
4,-0.548854


### 2.2、读取Microsoft Excel文件

In [57]:
#使用ExcelFile产生一个文件实例，再读取
xlsx=pd.ExcelFile('example/ex1.xlsx')
pd.read_excel(xlsx,'Sheet1',index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [59]:
#也可以直接用pd.read_excel读取
frame=pd.read_excel('example/ex1.xlsx','Sheet1',index_col=0)
print(frame)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [62]:
#写入时先使用ExcelWriter产生一个文件实例，再写入
writer=pd.ExcelWriter('example/ex2.xlsx')
frame.to_excel(writer,'Sheet1')
writer.close()

In [64]:
#也可以直接用to_excel
frame.to_excel('example/ex2.xlsx','Sheet2')

## 3、与Web API交互

In [68]:
#利用requests包进行
import requests
url='https://api.github.com/repos/pandas-dev/pandas/issues'
resp=requests.get(url)
print(type(resp))

<class 'requests.models.Response'>


In [67]:
data=resp.json()
data[0]['title']
print(type(data))

<class 'list'>


In [69]:
issues=pd.DataFrame(data,columns=['number','title','labels','state'])
print(issues)

    number                                              title  \
0    55195                                              BUG:    
1    55194  BUG: `DataFrame.var()` with `axis=1` returns `...   
2    55193  TST: Test GroupBy.__getitem__ with a column fr...   
3    55192  CLN: do not import Axes/Artist/Figure from mat...   
4    55190                 TYP: Allow None in Period.strftime   
..     ...                                                ...   
25   55152                     CLN: Add is_hashable_non_slice   
26   55151            Refactor npy_datetimestruct_to_datetime   
27   55150                  Replace cpplint with clang-format   
28   55147     DEPR/API: Series[categorical].replace behavior   
29   55145      ENH: pearson correlation with population mean   

                                               labels state  
0   [{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...  open  
1   [{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...  open  
2                                

## 4、与数据库的交互

In [ ]:
#略